In [1]:
%%capture
!pip install pandas scikit-learn xgboost shap joblib streamlit pyngrok
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torch-geometric -f https://data.pyg.org/whl/torch-2.2.0+cpu.html

import os, random, numpy as np, torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("✅ Dependencies installed and seeds set.")


In [2]:
import pandas as pd
from datetime import datetime, timedelta

os.makedirs("data", exist_ok=True)

def generate_developers(num_devs=8):
    # Simple dev pool with primary skills
    first_names = ["Aarav", "Neha", "Rohan", "Isha", "Kunal", "Meera", "Vikram", "Sara", "Akash", "Priya"]
    last_names = ["Sharma", "Patel", "Verma", "Iyer", "Rao", "Singh", "Nair", "Kapoor"]
    tech_stack = ["backend", "frontend", "mobile", "devops", "data", "qa"]
    devs = []
    for i in range(num_devs):
        name = f"{random.choice(first_names)} {random.choice(last_names)}"
        primary_skill = random.choice(tech_stack)
        devs.append((f"DEV_{i+1}", name, primary_skill))
    return pd.DataFrame(devs, columns=["developer_id", "developer_name", "primary_skill"])

def generate_sprint_tasks(dev_df, num_sprints=4, tasks_per_sprint=25, start_date=None):
    # Generate a simple Scrum-style task dataset
    if start_date is None:
        start_date = datetime(2025, 1, 1)
    statuses = ["Done", "In Progress", "To Do", "Blocked"]
    complexity_levels = [1, 2, 3, 5, 8, 13]
    task_types = ["feature", "bug", "chore"]
    components = ["backend", "frontend", "api", "database", "infra", "testing"]
    rows = []
    task_id_counter = 1
    for sprint in range(1, num_sprints + 1):
        sprint_name = f"Sprint {sprint}"
        sprint_start = start_date + timedelta(days=(sprint - 1) * 14)
        sprint_end = sprint_start + timedelta(days=13)
        for _ in range(tasks_per_sprint):
            task_id = f"TASK-{task_id_counter}"
            task_type = random.choice(task_types)
            component = random.choice(components)
            story_points = random.choice(complexity_levels)

            # Infer primary skill needed
            if component in ["backend", "api", "database"]:
                needed_skill = "backend"
            elif component == "frontend":
                needed_skill = "frontend"
            elif component == "infra":
                needed_skill = "devops"
            elif component == "testing":
                needed_skill = "qa"
            else:
                needed_skill = random.choice(["backend", "frontend", "devops", "qa"])

            # Prefer devs whose primary skill matches component
            candidate_devs = dev_df[dev_df["primary_skill"] == needed_skill]
            if candidate_devs.empty:
                dev_row = dev_df.sample(1).iloc[0]
            else:
                dev_row = candidate_devs.sample(1).iloc[0]

            developer_id = dev_row["developer_id"]
            developer_name = dev_row["developer_name"]

            expected_days = max(1, int(story_points / 2))
            base_days = max(1, expected_days + np.random.randint(-1, 3))

            # Introduce some delays
            delay_flag = np.random.rand() < 0.3
            if delay_flag:
                actual_days = base_days + np.random.randint(1, 4)
            else:
                actual_days = base_days

            created_date = sprint_start + timedelta(days=np.random.randint(0, 5))
            completed_date = created_date + timedelta(days=actual_days)

            if completed_date > sprint_end + timedelta(days=3):
                status = random.choice(["In Progress", "Blocked"])
            else:
                status = random.choice(["Done", "Done", "Done", "In Progress", "Blocked", "To Do"])

            blocker_flag = 1 if status == "Blocked" or np.random.rand() < 0.15 else 0
            if task_type == "bug":
                reopen_count = np.random.choice([0, 0, 0, 1, 2])
            else:
                reopen_count = np.random.choice([0, 0, 1])

            rows.append((
                task_id, sprint_name, task_type, component, story_points,
                needed_skill, developer_id, developer_name,
                expected_days, actual_days, delay_flag, status,
                blocker_flag, reopen_count, created_date.date(), completed_date.date()
            ))
            task_id_counter += 1

    columns = [
        "task_id", "sprint_name", "task_type", "component", "story_points",
        "primary_skill_needed", "developer_id", "developer_name",
        "expected_days", "actual_days", "delay_flag", "status",
        "blocker_flag", "reopen_count", "created_date", "completed_date",
    ]
    return pd.DataFrame(rows, columns=columns)

df_devs = generate_developers(num_devs=8)
df_tasks = generate_sprint_tasks(df_devs, num_sprints=4, tasks_per_sprint=25)

df_devs.to_csv("data/developers.csv", index=False)
df_tasks.to_csv("data/synthetic_tasks.csv", index=False)

print("Developers:", df_devs.shape)
print("Tasks:", df_tasks.shape)
df_tasks.head()


Developers: (8, 3)
Tasks: (100, 16)


,task_id,sprint_name,task_type,component,story_points,primary_skill_needed,developer_id,developer_name,expected_days,actual_days,delay_flag,status,blocker_flag,reopen_count,created_date,completed_date
0,TASK-1,Sprint 1,bug,infra,3,devops,DEV_2,Kunal Iyer,1,2,False,Done,0,2,2025-01-03,2025-01-05
1,TASK-2,Sprint 1,feature,testing,5,qa,DEV_1,Neha Sharma,2,4,False,Done,0,0,2025-01-05,2025-01-09
2,TASK-3,Sprint 1,bug,frontend,2,frontend,DEV_2,Kunal Iyer,1,3,False,Done,0,2,2025-01-01,2025-01-04
3,TASK-4,Sprint 1,feature,backend,5,backend,DEV_5,Vikram Sharma,2,7,True,Done,0,1,2025-01-03,2025-01-10
4,TASK-5,Sprint 1,bug,api,8,backend,DEV_5,Vikram Sharma,4,6,True,Done,0,1,2025-01-03,2025-01-09


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

def add_task_level_features(df_tasks: pd.DataFrame) -> pd.DataFrame:
    df = df_tasks.copy()
    df["expected_days"] = df["expected_days"].astype(int)
    df["actual_days"] = df["actual_days"].astype(int)
    df["delay_days"] = df["actual_days"] - df["expected_days"]
    df["is_delayed"] = (df["delay_days"] > 0).astype(int)
    return df

def add_developer_aggregates(df_tasks: pd.DataFrame) -> pd.DataFrame:
    df = df_tasks.copy()
    dev_stats = df.groupby("developer_id").agg(
        dev_avg_delay_days=("delay_days", "mean"),
        dev_avg_reopen_count=("reopen_count", "mean"),
        dev_task_count=("task_id", "count"),
        dev_delay_rate=("is_delayed", "mean"),
    ).reset_index()
    df = df.merge(dev_stats, on="developer_id", how="left")
    return df

def encode_categoricals(df: pd.DataFrame, categorical_cols):
    # NOTE: sparse_output=False for newer sklearn
    enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    cat_data = enc.fit_transform(df[categorical_cols])
    cat_cols = enc.get_feature_names_out(categorical_cols)
    df_cat = pd.DataFrame(cat_data, columns=cat_cols, index=df.index)
    df_num = df.drop(columns=categorical_cols)
    df_encoded = pd.concat([df_num, df_cat], axis=1)
    return df_encoded, enc

df_tasks_proc = add_task_level_features(df_tasks)
df_tasks_proc = add_developer_aggregates(df_tasks_proc)

target_col = "is_delayed"
feature_cols = [
    "story_points", "expected_days", "actual_days", "delay_days",
    "blocker_flag", "reopen_count",
    "dev_avg_delay_days", "dev_avg_reopen_count", "dev_task_count", "dev_delay_rate",
]
categorical_cols = ["task_type", "component", "sprint_name", "primary_skill_needed", "status"]

cols_to_keep = feature_cols + categorical_cols + [target_col, "task_id"]
df_model = df_tasks_proc[cols_to_keep].copy()

df_features = df_model.drop(columns=[target_col, "task_id"])
df_encoded, enc = encode_categoricals(df_features, categorical_cols=categorical_cols)

df_encoded[target_col] = df_model[target_col].values
df_encoded["task_id"] = df_model["task_id"].values

os.makedirs("data/processed", exist_ok=True)
df_encoded.to_csv("data/processed/tasks_processed.csv", index=False)

X = df_encoded.drop(columns=[target_col, "task_id"])
y = df_encoded[target_col]

X_train, X_test, y_train, y_test, task_id_train, task_id_test = train_test_split(
    X, y, df_encoded["task_id"], test_size=0.2, random_state=42, stratify=y
)

print("X_train:", X_train.shape, "X_test:", X_test.shape)
df_encoded.head()


X_train: (80, 31) X_test: (20, 31)


,story_points,expected_days,actual_days,delay_days,blocker_flag,reopen_count,dev_avg_delay_days,dev_avg_reopen_count,dev_task_count,dev_delay_rate,...,primary_skill_needed_backend,primary_skill_needed_devops,primary_skill_needed_frontend,primary_skill_needed_qa,status_Blocked,status_Done,status_In Progress,status_To Do,is_delayed,task_id
0,3,1,2,1,0,2,1.066667,0.600000,15,0.600000,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1,TASK-1
1,5,2,4,2,0,0,1.000000,0.312500,16,0.625000,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,TASK-2
2,2,1,3,2,0,2,1.066667,0.600000,15,0.600000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,TASK-3
3,5,2,7,5,0,1,1.408163,0.591837,49,0.714286,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,TASK-4
4,8,4,6,2,0,1,1.408163,0.591837,49,0.714286,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,TASK-5


In [4]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import joblib
import shap

os.makedirs("models", exist_ok=True)

xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
try:
    roc_auc = roc_auc_score(y_test, y_proba)
except ValueError:
    roc_auc = None

print("✅ XGBoost trained")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("Classification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

joblib.dump(xgb_model, "models/risk_xgb_model.pkl")
print("✅ Model saved to models/risk_xgb_model.pkl")

explainer = shap.TreeExplainer(xgb_model)

def explain_task_inline(task_id: str, max_display: int = 10):
    if task_id not in df_encoded["task_id"].values:
        raise ValueError(f"Task {task_id} not found.")
    feature_cols = [c for c in df_encoded.columns if c not in ["is_delayed", "task_id"]]
    row = df_encoded[df_encoded["task_id"] == task_id]
    X_row = row[feature_cols]
    shap_vals = explainer.shap_values(X_row)[0]
    risk_score = float(xgb_model.predict_proba(X_row)[:, 1][0])
    pred_label = int(xgb_model.predict(X_row)[0])
    abs_vals = np.abs(shap_vals)
    sorted_idx = np.argsort(abs_vals)[::-1]
    feature_importance = []
    for idx in sorted_idx[:max_display]:
        feature_importance.append({
            "feature": feature_cols[idx],
            "shap_value": float(shap_vals[idx]),
            "abs_shap": float(abs_vals[idx])
        })
    return {
        "task_id": task_id,
        "prediction": pred_label,
        "risk_score": risk_score,
        "feature_importance": feature_importance
    }

def generate_risk_explanation(task_row: dict, shap_info: dict) -> str:
    task_id = task_row.get("task_id")
    sprint = task_row.get("sprint_name")
    dev_name = task_row.get("developer_name")
    story_points = task_row.get("story_points")
    component = task_row.get("component")
    status = task_row.get("status")
    reopen_count = task_row.get("reopen_count")
    blocker_flag = task_row.get("blocker_flag")

    risk_score = shap_info["risk_score"]
    prediction = shap_info["prediction"]
    fi = shap_info["feature_importance"]

    if risk_score >= 0.8:
        risk_level = "very high"
    elif risk_score >= 0.6:
        risk_level = "high"
    elif risk_score >= 0.4:
        risk_level = "moderate"
    else:
        risk_level = "low"

    top_reasons = []
    for item in fi[:5]:
        feat = item["feature"]
        val = item["shap_value"]
        direction = "increases" if val > 0 else "reduces"
        top_reasons.append(f"- {feat} {direction} the delay risk")
    reasons_text = "\n".join(top_reasons) if top_reasons else "- Model features not available."

    recs = []
    if story_points and story_points >= 8:
        recs.append("Consider splitting this task into smaller stories to reduce complexity.")
    if blocker_flag == 1:
        recs.append("Unblock dependencies or clarify requirements before continuing.")
    if reopen_count and reopen_count > 1:
        recs.append("Review previous fixes or tests, as multiple reopen events suggest instability.")
    if status in ["Blocked", "In Progress"] and risk_score >= 0.6:
        recs.append("Re-evaluate ownership or add support from an experienced developer.")
    if not recs:
        recs.append("Monitor the task but no immediate intervention seems necessary.")

    lines = []
    lines.append(f"Task: {task_id} (Sprint: {sprint}, Component: {component})")
    lines.append(f"Assigned to: {dev_name}")
    lines.append(f"Status: {status}")
    lines.append(f"Story points: {story_points}")
    lines.append(f"Reopen count: {reopen_count}, Blocker flag: {blocker_flag}")
    lines.append("")
    lines.append("Model prediction:")
    lines.append(f"- Predicted delay risk: {risk_score:.2f} ({risk_level})")
    lines.append(f"- Predicted label: {'Delayed' if prediction == 1 else 'On-time'}")
    lines.append("")
    lines.append("Why the model thinks this task is risky:")
    lines.append(reasons_text)
    lines.append("")
    lines.append("Suggested actions:")
    for r in recs:
        lines.append(f"- {r}")
    return "\n".join(lines)

example_task = df_tasks.iloc[0]["task_id"]
print("Example task:", example_task)
shap_info = explain_task_inline(example_task)
task_row = df_tasks[df_tasks["task_id"] == example_task].iloc[0].to_dict()
print(generate_risk_explanation(task_row, shap_info))


✅ XGBoost trained
Accuracy: 1.0
ROC-AUC: 1.0
Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        13

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Confusion matrix:
 [[ 7  0]
 [ 0 13]]
✅ Model saved to models/risk_xgb_model.pkl
Example task: TASK-1
Task: TASK-1 (Sprint: Sprint 1, Component: infra)
Assigned to: Kunal Iyer
Status: Done
Story points: 3
Reopen count: 2, Blocker flag: 0

Model prediction:
- Predicted delay risk: 0.98 (very high)
- Predicted label: Delayed

Why the model thinks this task is risky:
- delay_days increases the delay risk
- story_points increases the delay risk
- actual_days reduces the delay risk
- blocker_flag increases the delay risk
- dev_avg_delay_days reduces the delay risk

Suggested actions:
- Review previous 

In [6]:
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
import torch.nn as nn
import torch.nn.functional as F

def build_bipartite_dev_task_graph(df_tasks, df_devs):
    dev_ids = df_devs["developer_id"].unique().tolist()
    task_ids = df_tasks["task_id"].unique().tolist()

    dev_id_to_idx = {d: i for i, d in enumerate(dev_ids)}
    task_id_to_idx = {t: i for i, t in enumerate(task_ids)}

    num_devs = len(dev_ids)
    num_tasks = len(task_ids)

    skill_types = df_devs["primary_skill"].unique().tolist()
    components = df_tasks["component"].unique().tolist()

    feature_dim = max(len(skill_types), 1 + len(components))
    if feature_dim < 4:
        feature_dim = 4

    # Dev features
    skill_to_idx = {s: i for i, s in enumerate(skill_types)}
    dev_x = torch.zeros((num_devs, feature_dim), dtype=torch.float)
    for _, row in df_devs.iterrows():
        d_idx = dev_id_to_idx[row["developer_id"]]
        s_idx = skill_to_idx[row["primary_skill"]]
        s_idx = min(s_idx, feature_dim - 1)
        dev_x[d_idx, s_idx] = 1.0

    # Task features
    comp_to_idx = {c: i for i, c in enumerate(components)}
    task_x = torch.zeros((num_tasks, feature_dim), dtype=torch.float)
    max_sp = max(df_tasks["story_points"].max(), 1)
    for _, row in df_tasks.iterrows():
        t_idx = task_id_to_idx[row["task_id"]]
        story_points = row["story_points"]
        comp = row["component"]
        comp_idx = comp_to_idx[comp]
        task_x[t_idx, 0] = float(story_points) / max_sp
        col = 1 + comp_idx
        if col >= feature_dim:
            col = feature_dim - 1
        task_x[t_idx, col] = 1.0

    edges_src, edges_dst = [], []
    for _, row in df_tasks.iterrows():
        dev_id = row["developer_id"]
        task_id = row["task_id"]
        d_idx = dev_id_to_idx[dev_id]
        t_idx = task_id_to_idx[task_id] + num_devs
        edges_src.append(d_idx)
        edges_dst.append(t_idx)
        edges_src.append(t_idx)
        edges_dst.append(d_idx)

    edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)
    x = torch.cat([dev_x, task_x], dim=0)

    print("dev_x:", dev_x.shape, "task_x:", task_x.shape, "x:", x.shape, "edge_index:", edge_index.shape)

    data = Data(x=x, edge_index=edge_index)
    data.num_devs = num_devs
    data.num_tasks = num_tasks
    data.dev_id_to_idx = dev_id_to_idx
    data.task_id_to_idx = task_id_to_idx
    data.dev_idx_to_id = {v: k for k, v in dev_id_to_idx.items()}
    data.task_idx_to_id = {v: k for k, v in task_id_to_idx.items()}
    return data

class DevTaskGraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels=32, out_channels=32):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

def train_unsupervised_embedding(data: Data, epochs: int = 20, lr: float = 1e-3):
    device = torch.device("cpu")
    data = data.to(device)
    model = DevTaskGraphSAGE(in_channels=data.x.size(-1)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        z = model(data.x, data.edge_index)
        loss = (z ** 2).mean()
        loss.backward()
        optimizer.step()
        if epoch in [1, 5, 10, 20]:
            print(f"Epoch {epoch}/{epochs} - loss: {float(loss.item()):.4f}")
    model.eval()
    with torch.no_grad():
        embeddings = model(data.x, data.edge_index).cpu()
    return model, embeddings

def recommend_devs_for_task(task_id: str, data: Data, embeddings, top_k: int = 3):
    num_devs = data.num_devs
    if task_id not in data.task_id_to_idx:
        raise ValueError(f"Task id {task_id} not found in graph.")
    task_local_idx = data.task_id_to_idx[task_id]
    task_global_idx = task_local_idx + num_devs
    task_emb = embeddings[task_global_idx]
    dev_embs = embeddings[:num_devs]
    task_norm = task_emb / (task_emb.norm() + 1e-8)
    dev_norms = dev_embs / (dev_embs.norm(dim=1, keepdim=True) + 1e-8)
    sims = torch.matmul(dev_norms, task_norm)
    sims_np = sims.numpy()
    top_idx = sims_np.argsort()[::-1][:top_k]
    results = []
    for idx in top_idx:
        dev_id = data.dev_idx_to_id[idx]
        score = float(sims_np[idx])
        results.append((dev_id, score))
    return results

graph_data = build_bipartite_dev_task_graph(df_tasks, df_devs)
gnn_model, gnn_embeddings = train_unsupervised_embedding(graph_data, epochs=10)
example_task_for_gnn = df_tasks.iloc[0]["task_id"]
print("GNN recommendations for:", example_task_for_gnn)
print(recommend_devs_for_task(example_task_for_gnn, graph_data, gnn_embeddings, top_k=3))

dev_x: torch.Size([8, 7]) task_x: torch.Size([100, 7]) x: torch.Size([108, 7]) edge_index: torch.Size([2, 200])
Epoch 1/10 - loss: 0.0697
Epoch 5/10 - loss: 0.0502
Epoch 10/10 - loss: 0.0328
GNN recommendations for: TASK-1
[('DEV_8', 0.8860174417495728), ('DEV_2', 0.8092111945152283), ('DEV_6', 0.7620481848716736)]


In [19]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import shap
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv

def load_data():
    df_tasks = pd.read_csv("data/synthetic_tasks.csv")
    df_devs = pd.read_csv("data/developers.csv")
    df_encoded = pd.read_csv("data/processed/tasks_processed.csv")
    return df_tasks, df_devs, df_encoded

def load_model_and_explainer(df_encoded):
    model = joblib.load("models/risk_xgb_model.pkl")
    feature_cols = [c for c in df_encoded.columns if c not in ["is_delayed", "task_id"]]
    explainer = shap.TreeExplainer(model)
    return model, explainer

def explain_task_inline_app(task_id: str, df_encoded, model, explainer, max_display: int = 10):
    if task_id not in df_encoded["task_id"].values:
        raise ValueError(f"Task {task_id} not found.")
    feature_cols = [c for c in df_encoded.columns if c not in ["is_delayed", "task_id"]]
    row = df_encoded[df_encoded["task_id"] == task_id]
    X_row = row[feature_cols]
    shap_vals = explainer.shap_values(X_row)[0]
    risk_score = float(model.predict_proba(X_row)[:, 1][0])
    pred_label = int(model.predict(X_row)[0])
    abs_vals = np.abs(shap_vals)
    sorted_idx = np.argsort(abs_vals)[::-1]
    feature_importance = []
    for idx in sorted_idx[:max_display]:
        feature_importance.append({
            "feature": feature_cols[idx],
            "shap_value": float(shap_vals[idx]),
            "abs_shap": float(abs_vals[idx])
        })
    return {
        "task_id": task_id,
        "prediction": pred_label,
        "risk_score": risk_score,
        "feature_importance": feature_importance
    }

def generate_risk_explanation_app(task_row: dict, shap_info: dict) -> str:
    task_id = task_row.get("task_id")
    sprint = task_row.get("sprint_name")
    dev_name = task_row.get("developer_name")
    story_points = task_row.get("story_points")
    component = task_row.get("component")
    status = task_row.get("status")
    reopen_count = task_row.get("reopen_count")
    blocker_flag = task_row.get("blocker_flag")

    risk_score = shap_info["risk_score"]
    prediction = shap_info["prediction"]
    fi = shap_info["feature_importance"]

    if risk_score >= 0.8:
        risk_level = "very high"
    elif risk_score >= 0.6:
        risk_level = "high"
    elif risk_score >= 0.4:
        risk_level = "moderate"
    else:
        risk_level = "low"

    top_reasons = []
    for item in fi[:5]:
        feat = item["feature"]
        val = item["shap_value"]
        direction = "increases" if val > 0 else "reduces"
        top_reasons.append(f"- {feat} {direction} the delay risk")
    reasons_text = "\n".join(top_reasons) if top_reasons else "- Model features not available."

    recs = []
    if story_points and story_points >= 8:
        recs.append("Consider splitting this task into smaller stories to reduce complexity.")
    if blocker_flag == 1:
        recs.append("Unblock dependencies or clarify requirements before continuing.")
    if reopen_count and reopen_count > 1:
        recs.append("Review previous fixes or tests, as multiple reopen events suggest instability.")
    if status in ["Blocked", "In Progress"] and risk_score >= 0.6:
        recs.append("Re-evaluate ownership or add support from an experienced developer.")
    if not recs:
        recs.append("Monitor the task but no immediate intervention seems necessary.")

    lines = []
    lines.append(f"Task: {task_id} (Sprint: {sprint}, Component: {component})")
    lines.append(f"Assigned to: {dev_name}")
    lines.append(f"Status: {status}")
    lines.append(f"Story points: {story_points}")
    lines.append(f"Reopen count: {reopen_count}, Blocker flag: {blocker_flag}")
    lines.append("")
    lines.append("Model prediction:")
    lines.append(f"- Predicted delay risk: {risk_score:.2f} ({risk_level})")
    lines.append(f"- Predicted label: {'Delayed' if prediction == 1 else 'On-time'}")
    lines.append("")
    lines.append("Why the model thinks this task is risky:")
    lines.append(reasons_text)
    lines.append("")
    lines.append("Suggested actions:")
    for r in recs:
        lines.append(f"- {r}")
    return "\n".join(lines)

def build_bipartite_dev_task_graph(df_tasks, df_devs):
    dev_ids = df_devs["developer_id"].unique().tolist()
    task_ids = df_tasks["task_id"].unique().tolist()
    dev_id_to_idx = {d: i for i, d in enumerate(dev_ids)}
    task_id_to_idx = {t: i for i, t in enumerate(task_ids)}
    num_devs = len(dev_ids)
    num_tasks = len(task_ids)

    skill_types = df_devs["primary_skill"].unique().tolist()
    components = df_tasks["component"].unique().tolist()
    feature_dim = max(len(skill_types), 1 + len(components))
    if feature_dim < 4:
        feature_dim = 4

    skill_to_idx = {s: i for i, s in enumerate(skill_types)}
    dev_x = torch.zeros((num_devs, feature_dim), dtype=torch.float)
    for _, row in df_devs.iterrows():
        d_idx = dev_id_to_idx[row["developer_id"]]
        s_idx = skill_to_idx[row["primary_skill"]]
        s_idx = min(s_idx, feature_dim - 1)
        dev_x[d_idx, s_idx] = 1.0

    comp_to_idx = {c: i for i, c in enumerate(components)}
    task_x = torch.zeros((num_tasks, feature_dim), dtype=torch.float)
    max_sp = max(df_tasks["story_points"].max(), 1)
    for _, row in df_tasks.iterrows():
        t_idx = task_id_to_idx[row["task_id"]]
        story_points = row["story_points"]
        comp = row["component"]
        comp_idx = comp_to_idx[comp]
        task_x[t_idx, 0] = float(story_points) / max_sp
        col = 1 + comp_idx
        if col >= feature_dim:
            col = feature_dim - 1
        task_x[t_idx, col] = 1.0

    edges_src, edges_dst = [], []
    for _, row in df_tasks.iterrows():
        dev_id = row["developer_id"]
        task_id = row["task_id"]
        d_idx = dev_id_to_idx[dev_id]
        t_idx = task_id_to_idx[task_id] + num_devs
        edges_src.append(d_idx)
        edges_dst.append(t_idx)
        edges_src.append(t_idx)
        edges_dst.append(d_idx)

    edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)
    x = torch.cat([dev_x, task_x], dim=0)

    data = Data(x=x, edge_index=edge_index)
    data.num_devs = num_devs
    data.num_tasks = num_tasks
    data.dev_id_to_idx = dev_id_to_idx
    data.task_id_to_idx = task_id_to_idx
    data.dev_idx_to_id = {v: k for k, v in dev_id_to_idx.items()}
    data.task_idx_to_id = {v: k for k, v in task_id_to_idx.items()}
    return data

class DevTaskGraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels=32, out_channels=32):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

def train_unsupervised_embedding(data: Data, epochs: int = 20, lr: float = 1e-3):
    device = torch.device("cpu")
    data = data.to(device)
    model = DevTaskGraphSAGE(in_channels=data.x.size(-1)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        z = model(data.x, data.edge_index)
        loss = (z ** 2).mean()
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        embeddings = model(data.x, data.edge_index).cpu()
    return model, embeddings

def recommend_devs_for_task_app(task_id: str, data: Data, embeddings, top_k: int = 3):
    num_devs = data.num_devs
    if task_id not in data.task_id_to_idx:
        raise ValueError(f"Task id {task_id} not found in graph.")
    task_local_idx = data.task_id_to_idx[task_id]
    task_global_idx = task_local_idx + num_devs
    task_emb = embeddings[task_global_idx]
    dev_embs = embeddings[:num_devs]
    task_norm = task_emb / (task_emb.norm() + 1e-8)
    dev_norms = dev_embs / (dev_embs.norm(dim=1, keepdim=True) + 1e-8)
    sims = torch.matmul(dev_norms, task_norm)
    sims_np = sims.numpy()
    top_idx = sims_np.argsort()[::-1][:top_k]
    results = []
    for idx in top_idx:
        dev_id = data.dev_idx_to_id[idx]
        score = float(sims_np[idx])
        results.append((dev_id, score))
    return results

df_tasks, df_devs, df_encoded = load_data()
xgb_model, shap_explainer = load_model_and_explainer(df_encoded)
feature_cols = [c for c in df_encoded.columns if c not in ["is_delayed", "task_id"]]
X_all = df_encoded[feature_cols]
risk_scores = xgb_model.predict_proba(X_all)[:, 1]
risk_labels = xgb_model.predict(X_all)

df = df_tasks.copy()
df["risk_score"] = risk_scores
df["risk_label"] = risk_labels

graph_data = build_bipartite_dev_task_graph(df_tasks, df_devs)
_, gnn_embeddings = train_unsupervised_embedding(graph_data, epochs=10)

st.set_page_config(page_title="AI PM Copilot", layout="wide")
st.title("AI Project Management Copilot")

st.sidebar.header("Filters")
sprints = sorted(df["sprint_name"].unique())
selected_sprints = st.sidebar.multiselect("Sprint", options=sprints, default=sprints)
devs = sorted(df["developer_name"].unique())
selected_devs = st.sidebar.multiselect("Developer", options=devs, default=devs)
statuses = sorted(df["status"].unique())
selected_status = st.sidebar.multiselect("Status", options=statuses, default=statuses)
risk_threshold = st.sidebar.slider("Minimum risk score", 0.0, 1.0, 0.5, 0.05)
show_only_high_risk = st.sidebar.checkbox("Show only high-risk tasks", value=False)

df_filtered = df[
    df["sprint_name"].isin(selected_sprints)
    & df["developer_name"].isin(selected_devs)
    & df["status"].isin(selected_status)
    & (df["risk_score"] >= risk_threshold)
]
if show_only_high_risk:
    df_filtered = df_filtered[df_filtered["risk_label"] == 1]

st.subheader("📊 Task Table")
st.write(f"Tasks in view: {len(df_filtered)}")
display_cols = [
    "task_id", "sprint_name", "task_type", "component",
    "story_points", "primary_skill_needed", "developer_name",
    "expected_days", "actual_days", "status",
    "blocker_flag", "reopen_count", "risk_score", "risk_label"
]
st.dataframe(df_filtered[display_cols].sort_values("risk_score", ascending=False), use_container_width=True)

st.subheader("🔍 Task Insight Panel")
if len(df_filtered) == 0:
    st.info("No tasks match the current filters.")
else:
    selected_task_id = st.selectbox(
        "Select a task for analysis",
        options=df_filtered["task_id"].unique().tolist(),
    )
    task_row = df[df["task_id"] == selected_task_id].iloc[0].to_dict()
    shap_info = explain_task_inline_app(selected_task_id, df_encoded, xgb_model, shap_explainer)
    explanation_text = generate_risk_explanation_app(task_row, shap_info)

    st.markdown("#### 🧠 Model Explanation")
    st.text_area("Why is this task risky?", value=explanation_text, height=260)

    st.markdown("#### 👥 Recommended Developers (GNN)")
    try:
        recs = recommend_devs_for_task_app(selected_task_id, graph_data, gnn_embeddings, top_k=3)
        rec_df = pd.DataFrame(recs, columns=["developer_id", "similarity"])
        dev_map = df_devs.set_index("developer_id")["developer_name"].to_dict()
        rec_df["developer_name"] = rec_df["developer_id"].map(dev_map)
        rec_df["similarity"] = rec_df["similarity"].round(3)
        st.dataframe(rec_df, use_container_width=True)
    except Exception as e:
        st.warning(f"GNN recommendations not available: {e}")


Overwriting app.py


In [14]:
!cat ~/.streamlit/logs/*.log


cat: '/root/.streamlit/logs/*.log': No such file or directory


In [17]:
from pyngrok import ngrok

# Kill any local streamlit or ngrok processes
!pkill -f streamlit || echo "no streamlit running"
ngrok.kill()


^C


In [20]:
from pyngrok import ngrok

!pkill -f streamlit || echo "no previous streamlit"
ngrok.kill()

ngrok.set_auth_token("35VDKWTfihG5nNoBkNGrJIfdt3b_2BtgM1pM8fedNt8vu5sKH")
get_ipython().system_raw('streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &')

public_url = ngrok.connect(8501, proto="http")
print(" Streamlit is live at:", public_url.public_url)


^C
🚀 Streamlit is live at: https://diencephalic-layton-unvouchsafed.ngrok-free.dev
